<a href="https://colab.research.google.com/github/HelloShagun/EOWater/blob/main/Map_IGARSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
pip install earthengine-api geemap mss cartopy

In [4]:
import ee
import geemap

# 1. Initialize with your specific Project ID
project_id = 'ee-shagungargc2020'

try:
    ee.Initialize(project=project_id)
    print("Earth Engine initialized successfully!")
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project=project_id)

# 2. Region of Interest: UK
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
uk = countries.filter(ee.Filter.eq('country_na', 'United Kingdom'))
uk_geom = uk.geometry()

# 3. Sentinel-2 Mosaic Logic
def mask_clouds(image):
    scl = image.select('SCL')
    # Keeping vegetation, soils, and water; masking clouds and shadows
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return image.updateMask(mask).divide(10000)

s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(uk_geom) \
    .filterDate('2024-03-01', '2024-09-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(mask_clouds)

s2_mosaic = s2_col.median().clip(uk_geom)


Earth Engine initialized successfully!


In [5]:


# 4. HydroLAKES Data
# Using the specific project path you provided
lakes = ee.FeatureCollection("projects/sat-io/open-datasets/HydroLakes/lake_poly_v10")
uk_lakes = lakes.filterBounds(uk_geom)

# 5. Visualization Setup
Map = geemap.Map(layout={'height': '800px', 'width': '100%'})
Map.centerObject(uk_geom, 6)

# Add the Satellite Background (B4, B3, B2 for True Color)
s2_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3, 'gamma': 1.3}
Map.addLayer(s2_mosaic, s2_vis, 'Sentinel-2 Mosaic')

# Add Lakes with a Highlighted Color (Bright Blue/Cyan)
Map.addLayer(uk_lakes, {'color': '00FFFF'}, 'HydroLakes (Highlighted)')

# Add UK Boundary Outline
outline = ee.Image().byte().paint(featureCollection=uk, color=1, width=2)
Map.addLayer(outline, {'palette': 'black'}, 'UK Boundary')

# 6. Final Map Display
Map
Map.to_image(filename=output_file, monitor=True)

NameError: name 'output_file' is not defined

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patches as mpatches
import geemap

# 1. Convert GEE Lakes to a format Matplotlib can use (GeoPandas)
# This allows us to plot the specific data you found in GEE
print("Downloading lake data for plotting...")
uk_lakes_gdf = geemap.ee_to_gdf(uk_lakes)

# 2. Setup the figure
fig = plt.figure(figsize=(10, 12))
projection = ccrs.TransverseMercator(central_longitude=-2)
ax = plt.axes(projection=projection)
ax.set_extent([-11, 2, 49.5, 61], crs=ccrs.PlateCarree())

# 3. Add Professional Backgrounds
# stock_img() provides a clean, fast-rendering global satellite-style background
ax.stock_img()

# Add high-resolution borders and coastlines
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5)
ax.add_feature(cfeature.BORDERS.with_scale('10m'), linestyle=':', linewidth=1)

# 4. Plot the HydroLAKES data
# We plot the GeoPandas dataframe directly onto the Cartopy axis
uk_lakes_gdf.plot(ax=ax, color='cyan', transform=ccrs.PlateCarree(), label='Lakes')

# 5. The "QGIS Frame": Gridlines and Labels
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 9, 'color': 'black'}
gl.ylabel_style = {'size': 9, 'color': 'black'}

# 6. Aesthetic Touches: Legend & North Arrow
lake_patch = mpatches.Patch(color='cyan', label='HydroLAKES')
border_patch = mpatches.Patch(edgecolor='black', facecolor='none', label='National Boundary')
plt.legend(handles=[lake_patch, border_patch], loc='lower right', frameon=True)

# Add a North Arrow
x, y, arrow_length = 0.05, 0.95, 0.08
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=3, headwidth=10),
            ha='center', va='center', fontsize=15, fontweight='bold',
            xycoords='axes fraction')

plt.title("Map of United Kingdom: Surface Water Features", pad=20, fontsize=14, fontweight='bold')

# 7. Save for Publication
output_path = '/content/UK_Hydro_Publication.png'
plt.savefig(output_path, bbox_inches='tight', dpi=300)
plt.show()

print(f"High-quality map saved to {output_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 69.7 MB/s eta 0:00:00
